In [1]:
# %pip install openai==1.12.0
# from notebookutils import mssparkutils
mssparkutils.session.restartPython()

StatementMeta(, , , Waiting, , Waiting)

In [ ]:
########################################################
############################ get taskID from environment
########################################################
df = spark.sql("SELECT * FROM InFreGen.task")
display(df)

task_row = spark.sql("SELECT * FROM InFreGen.task WHERE taskID = {}".format(taskID)).collect()[0]
print(task_row)


StatementMeta(, , , Waiting, , Waiting)

Row(taskID=14, userInput='I want a pic of sheep', keyword='sheep', num=5, resolution='1024*1024', sizeChoice='small', State=1)


In [ ]:
import os
from openai import AzureOpenAI
import json
ENDPOINT = "https://mango-bush-0a9e12903.5.azurestaticapps.net/api/v1"
API_KEY = "ADD YOUR KEY HERE"

API_VERSION = "2024-02-01"
MODEL_NAME = "gpt-4o"

client = AzureOpenAI(
    azure_endpoint=ENDPOINT,
    api_key=API_KEY,
    api_version=API_VERSION,
)

user_input = {
    "input": task_row["userInput"],
}


MESSAGES = [
    {"role": "system", "content": "You are a helpful assistant for image searching."},
    {"role": "user", "content": "Generate a json file that includes the following details:"},
    {"role": "assistant", "content": "Please provide the necessary details for the json output."},
    {
        "role": "user",
        "content": f"Keywords: Based on the user's input: {user_input['input']}, generate diverse and appropriate keywords that describe the image needs.\n"
                "Make the keywords in varied and specific based on user input.\n"
                "For example, if the input is 'cat', generate keywords like 'Black cat, White cat, Tabby cat, Siamese cat, Calico cat, Maine Coon, Bengal cat, Persian cat, Sphynx cat, Ragdoll cat, Tuxedo cat, Ginger cat, British Shorthair, Scottish Fold, Norwegian Forest Cat, Himalayan cat, Abyssinian cat, Russian Blue'.\n"
                "If the input is 'city landscape,' generate keywords such as:\n"
                "'night cityscape, skyscrapers, urban skyline, street view, historic district, modern architecture, parks, monuments, riverfront skyline, busy traffic, aerial cityscape, downtown buildings.'\n\n"
                "Output should be in JSON format, structured as:\n"
                "If the input is 'medical image,' include keywords that specify types of medical imaging and body areas, such as:\n"
                "'CT scan, X-ray image, MRI, ultrasound, PET scan, chest X-ray, abdominal CT, brain MRI, bone density scan, radiology, thoracic CT, mammography, cardiac MRI, lung X-ray.'\n\n"
                "The number of generated keywords should equal 'Num' and allow repetition.\n"
                "Only return the keywords, in the format like A,B,C,D \n"
                "Return ONLY the JSON, no additional text. Do not include the 'json' in the head of the result"

    }
]


completion = client.chat.completions.create(
    model=MODEL_NAME,
    messages=MESSAGES,
)

completion_json = json.loads(completion.model_dump_json())  # Deserialize the JSON string into a dictionary
message_content = completion_json["choices"][0]["message"]["content"]
print(message_content)

In [ ]:
keyword = completion_json["choices"][0]["message"]["content"]
keyword = json.loads(keyword)

StatementMeta(, , , Waiting, , Waiting)

In [ ]:
# write the message_content to the keyword column in the task table
original_keyword = keyword['Keywords']
# write the original_keyword to the keyword column in the task table
# get the taskID from the task_row
taskID = task_row['taskID']
spark.sql("UPDATE InFreGen.task SET keyword = '{}' WHERE taskID = {}".format(original_keyword, taskID))

StatementMeta(, , , Waiting, , Waiting)

In [ ]:
df = spark.sql("SELECT * FROM InFreGen.task LIMIT 1000")
display(df)

StatementMeta(, , , Waiting, , Waiting)